# Exercicio 2

### Com o objetivo de desenvolver uma key encapsulation mechanism baseado no RSA era necessário desde logo perceber o funcionamento do RSA, acompanhadas de funções de encapsulamento e desencapsulamento de chaves simétricas a partir de chaves publicas e privadas, geradas pelos parâmetros definidos no RSA - esquema assimétrico de criptografia.
<br>
### Dada esta imposição foi necessário anuir alguns factos.

## Em primeiro lugar espera-se a geração de uma chave pública e privada. <br>
### Estas chaves têm parâmetros definidos, dos quais a mesma depende para a sua geração.
#### Assim : <br>
* n é o parâmetro de segurança
* Chave pública é definida por (n,e)
* Chave privada definida por (p,q,d)
<br>
#### Tendo isto em conta, e consultando os apontamentos do docente e de alguns documentos online
* Gera-se um número primo aleatório para o parâmetro q do RSA
* Gera-se um número primo aleatório para o parâmetro p do RSA
* O parâmetro n é calculado pela expressão n= p * q
* O phi, que permite calcular o expoente da chave pública é calculado por phi(n) = (p-1) * (q-1)
<br>
#### De forma a calcular o parametro d é necessário fazer algumas contas auxiliares
* Escolher um inteiro "e" entre 1 e phi(n) em que "e" e "phi(n) sejam relativamente primos
* Calcular o parâmetro auxiliar "d" tendo como base o algoritmo de euclides associado à formula d * e - k * phi(n) = 1
* Visto que é necessário descobrir um "k" e um "d" recorremos à identidade de bézout: gcd(x,y) = sx + ty a qual existe no sagemath com o nome xgcd(x,y) retornando um triplo (g,s,t) . A partir daqui infere-se que d será o resultado de d = mod( s,phi(n)), visto que 1 < d < phi(n)

## Encapsulamento e geração da chave
<br>
Recebe-se a chave pública como input (e,n) e para gerar uma chave simétrica devidamente encapsulada foi necessário:
* gerar um número inteiro aleatório entre 1 n-1
* calcular o encapsulamente da chave através da atribuição c <- r^e mod n
* gerar a chave simetrica a partir da atribuição do valor resultante de uma KDF aplicado ao parâmetro r. Entendemos usar um KDF com uma função de hash SHA-256, sendo dessa forma necessário adicionar um salt! Isto levará a que tenhamos como resultado o par (w,salt + c)

## Desencapsulamento da chave
<br>
Recebemos como argumento o resultado do encapsulamento da chave, contendo basicamente o (salt + c). De seguida:
* Obtém-se o valor de r através de:
    * r <- c^d mod n
* O r permite-nos gerar a chave simétrica usando mais uma vez um kdf cujo resultado é atribuido ao parametro w:
    * r <- KDF(r) 

In [1]:

import math
import os, hashlib



class KEM_RSA(object):

    def __init__(self,sec_par):
        
        # Mersenne numbers 
        # If p is prime and Mp=2p−1 is also prime,
        #  then Mp is called a Mersenne prime
        
        self.q = random_prime(pow(2,sec_par-1),pow(2,sec_par)-1)
        self.p = random_prime(pow(2,sec_par+1-1),pow(2,sec_par+1)-1)
        self.n = self.q * self.p

        
        self.phi = (self.p-1)*(self.q-1)

        self.e = ZZ.random_element(self.phi)
        
        while gcd(self.e, self.phi) != 1:
            self.e = ZZ.random_element(self.phi)
            
       
        self.bezout = xgcd(self.e, self.phi)
        
        

       
        s = self.bezout[1]
        self.d = Integer(mod(s, self.phi))




    def encapsula(self, e, n):
        
       
        r = ZZ.random_element(n)
        
       
        salt = os.urandom(16)

      
        key_encapsulation = Integer(power_mod(r, e, n))


        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)


        k = (w,salt + str(key_encapsulation).encode())       

        return k    
    
   
    def revelacao(self, cs):
        
       
        salt = cs[:16]
        c = int(cs[16:].decode())
        

        r = Integer(power_mod(c, self.d, self.n))
        

        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)
        
        return w



In [2]:
N = 1024

# Chave publica: (n,e)
# Chave privada: (p,q,d)
# Inicializacao da classe responsavel por implementar o KEM-RSA
kemrsa = KEM_RSA(N)

# Verificar que ed == 1 (mod φ(n))
#print(mod(kemrsa.e * kemrsa.d, kemrsa.phi))

# Procede-se ao encapsulamento
(w,c) = kemrsa.encapsula(kemrsa.e, kemrsa.n)

print("Chave devolvida pelo encapsulamento: ")
print(w)
print("\n'Encapsulamento' da chave: ")
print(c)

# Procede-se ao desencapsulamento
w1 = kemrsa.revelacao(c)
print("\nChave devolvida pelo desencapsulamento: ")
print(w1)

# Verificar se a chave devolvida pelo desencapsulamento é igual à que foi gerada no encapsulamento
if w == w1:
    print("OKAY, as chaves são iguais!!!")
else:
    print("NOP, as chaves são diferentes!!!")


Chave devolvida pelo encapsulamento: 
b'E\xa8E\x1eZ\xfe\xcb\xbb\x80V\xdd8\xf2>\x92g\xce\xf57\x84\xa2\x7f0\xc5\x83\x8e\x07\xd0\xd9\xb9\xeeD'

'Encapsulamento' da chave: 
b'"\xc7\xec,\xf8p\xe6X\xac\xbf\xe9\x93+%\xaaT601267022958762038824168982448452183842633205017389476598676485203039655439199198459467884652746706935343668095156574088211400923287539990263395962575774082666305076714243128979636603560495740782520855091081250251152157257073539200119763433808984211183148743753233400775330601645947328884621690719432662258602169960252293521834117873021102737243843683555461601596847127432399313930451926773171612196005633994304608107959589803436873443518542834240949732872233481941017205263705451206373517535856325193699096035722843754845007204083091234938750457164311345499410087203562272633632418843138017866015141198335706086825'

Chave devolvida pelo desencapsulamento: 
b'E\xa8E\x1eZ\xfe\xcb\xbb\x80V\xdd8\xf2>\x92g\xce\xf57\x84\xa2\x7f0\xc5\x83\x8e\x07\xd0\xd9\xb9\xeeD'
OKAY, as chaves são iguai

# Construir,  a partir deste KEM e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.

Como indicado no enunciado, usamos a classe **KEM-RSA** criada anteriormente para a geração das chaves pública e privada. 
<br>
<br>
A inicialização desta nova classe **PKE\_IND\_CCA** é, então, também a inicialização da classe **KEM\_RSA** com um determinado N (que determina o tamanho dos números primos p e q no RSA) de forma a, posteriormente, obter as chaves. 
<br>
Definimos também uma função para cifra e outra para decifra, para as quais necessitamos de umas outras auxiliares - $d$, $h$, $xor$ - que vamos descrever de seguida:
<br>
<br>
A função **_h_** gera um inteiro aleatório entre 0 e n, sendo que n é recebido pela função como argumento.
<br>
A função **_f_** gera uma chave e o seu encapsulamento, precisando, para isso, de receber como argumento o seguinte:
- a _seed_ para passar à KDF;
- um _salt_ para a KDF;
- a chave pública (e, n).

Por fim, a função **_xor_**, como o nome indica, realiza a operação binária de _exclusive or_ entre dois _arrays_ de bytes, byte a byte.
<br>
<br>
Finalmente, passamos a explicar as funções de cifra e decifra.
<br>
<br>
A função **_cifra_** é, como sugerido pelo nome, usada para cifrar uma mensagem. Então, esta função recebe como argumento a mensagem a cifrar e uma chave pública (e, n) a ser usada para a cifragem e efetua os seguintes passos:
- utiliza a função auxiliar $h$ para gerar um número inteiro aleatório $r$ em que $0 < r < n$;
- calcula o SHA-256 do número $r$ gerado, colocando o resultado em $g$;
- efetua o XOR entre a mensagem a cifrar e $g$, com recurso à função auxiliar $xor$ e guarda em $y$;
- é calculado $yi$, que é o _hash_ de $y$;
- é gerada uma chave $k$ e o seu encapsulamento $w$, usando a função $f$ com os seguintes parâmetros:
    + a _seed_ é a concatenação de $r$ com $yi$;
    + $e$ e $n$ são o (e, n) recebidos como parâmetro;
    + $salt$ é um conjunto de 16 bytes gerados aleatoriamente.
- calcula o XOR entre $r$ e $k$ e coloca em $c$;
- finalmente, devolve $y$ - a ofuscação da mensagem, $w$ - a chave encapsulada, $c$ - a ofuscação da chave.

A função **_decifra_** recebe como argumentos os resultados da cifra, a mensagem ofuscada $y$, a chave encapsulada $w$ e a chave ofuscada $c$. O procedimento de decifra é o seguinte:
- desencapsular a chave $w$ recorrendo à função da classe KEM-RSA e colocar em $k$;
- calcular o XOR entre $k$ e $c$;
- obter o _salt_ nos 16 primeiros bytes da chave $w$;
- calcular $yi$, o _hash_ de $y$;
- utilizar a função $f$ para gerar uma chave e o seu encapsulamento a partir dos parâmetros $y$ e $r$;
- testar se o resultado da opração anterior é igual a $(k, w)$;
- se sim:
    + calcula $g(r)$, sendo $g$ a função de hash SHA-256 ;
    + calcula o XOR entre $y$ e $g(r)$.
- se não, é gerada uma exceção.

In [3]:
import os, hashlib
# The Fujisaki-Okamoto (FO) transformation (CRYPTO 1999 and Journal of Cryptology 2013)
# turns any weakly secure public-key encryption scheme into a strongly (IND-CCA) secure one in
# the random oracle model. 

# Basicamente usa-se a KEM_RSA para gerar as chaves publica e privada
# ind-cca é Chosen Ciphertext Attack


class PKE_IND_CCA(object):
   
    def __init__(self, N):
        # N é o tamanho usado para os primos p e q no RSA
        self.kem = KEM_RSA(N)
    

    def h(self, n):
        # Gerar um inteiro aleatorio r tal que 0 < r < n 
        r = ZZ.random_element(n)
        return r
    
    
    # Funcao que gera a chave e o seu encapsulamento recebendo como parametro:
    # a seed do KDF,
    # a chave publica (e,n) e
    # um salt usado no KDF
    def f(self, seed, e, n, salt):
        # Criptograma usado para o encapsulamento da chave (c = seed^e mod n)
        c = Integer(power_mod(int(seed.decode()), e, n))
        
        # Geracao da chave simetrica a partir do seed (W = KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', seed, salt, 100000)
        
        return (w, salt + str(c).encode())

    
    # XOR de 2 arrays de bytes, byte-a-byte! 
    # A mensagem(data) deve ser menor ou igual á chave(mask)! 
    # Caso contrario, a chave ou mask é 'repetida' para os bytes seguintes dos dados
    def xor(self, data, key):
        masked = b''
        ldata = len(data)
        lmask = len(key)
        i = 0
        while i < ldata:
            for j in range(lmask):
                if i < ldata:
                    masked += (data[i] ^^ key[j]).to_bytes(1, byteorder='big')
                    i += 1
                else:
                    break  
        return masked
    
    
    # Funcao usada para cifrar; recebe a mensagem m e uma chave publica (e,n)
    def cifra(self, m, e, n):
        # Gerar um inteiro aleatorio r tal que 0 < r < n
        r = self.h(n)
        # Calculo do g(r), em que g é uma função de hash (no nosso caso, sha-256)
        g = hashlib.sha256(str(r).encode()).digest()
        # Efetuar o calculo do XOR entre: x e g(r)
        y = self.xor(m, g)
        yi = Integer('0x' + hashlib.sha256(y).hexdigest())
        # Gerar o salt para derivar a chave
        salt = os.urandom(16)
        # Calcular (k,w)
        (k,w) = self.f(str(yi + r).encode(), e, n, salt)
        # Calcular o XOR entre k e r
        c = self.xor(str(r).encode(), k)
        
        return (y,w,c)
    
    
    # Funcao usada para decifrar; recebe a mensagem ofuscada, o encapsulamento da chave e a chave ofuscada 
    def decifra(self, y, w, c):
        # Fazer o desencapsulamento da chave
        k = self.kem.revelacao(w)
        # Calcula o XOR entre c  e  k
        r = self.xor(c, k)
        # Buscar os 16 primeiros bytes de w para obter o salt
        salt = w[:16]
        yi = Integer('0x' + hashlib.sha256(y).hexdigest())
        # Verificar se (w,k) != f(y+r)
        if (k,w) != self.f(str(yi + int(r)).encode(), self.kem.e, self.kem.n, salt):
            # Lancar excecao
            raise IOError
        else:
            # Calculo do g(r), em que g é uma função de hash (no nosso caso, sha-256)
            g = hashlib.sha256(r).digest()
            # Calcular o XOR entre y e g(r)
            m = self.xor(y, g)
        
        return m

In [4]:
pke = PKE_IND_CCA(1024)

#message = os.urandom(32)
message = b"TP1 de Estruturas Criptograficas"

print(b"Mensagem original: " + message)

(y,w,c) = pke.cifra(message, pke.kem.e, pke.kem.n)

print("\nTexto cifrado:")
print(b"Y = " + y)
print(b"W = " + w)
print(b"C = " + c)

print("\nTexto decifrado:")

try:
    message1 = pke.decifra(y,w,c)
    print(b"Texto decifrado: " + message1)
    if message == message1:
        print("[A mensagem decifrada é igual à original!!!!]")
    else:
        print("[A mensagem decifrada é diferente da original!!!!]")
except IOError as e:
    print("Erro ao decifrar a mensagem!!!!")

b'Mensagem original: TP1 de Estruturas Criptograficas'

Texto cifrado:
b'Y = X\xc1MVX\x17\xdd\xc1\x0f\x88n:"V%\xac;\x859\xab\xa8\xa3\xa00\x1a\x02\x89_\xb8\xc6{\xd2'
b'W = a\xb6\x974\xab\xbd\xa0\xe5\xf32\xd9\x85\x98H\xbd\xc5759454101479038445443032318702535081912157674102348083360878591822684282765127485184033987450278751516032767720734066319741745573383527155598760224591845991487827693403885775699672620261843622578167852834471589826101118695074470811660672702373863915656956237043083835900975058296916996092337291089090931756933091429284336900159947681619594917110744146640971860566388087192256555219751059164986652734852437886239531552797996004968033356398166771171618437259445589932553115715027561784931344621407967580118922512127553758454855352157465268845499408229813768593944245821701874044074987092883243926893340384330875776106'
b'C = Q\xa9F\xe9:=#em\xb6\xe5\x06\x9b\xea\x8fJ\xe0\xa3`\x0f\x17o\x9c\xd8\xf1\xbf\xe9\xb4\xd6{\xf4.^\xadA\xe9;;+eh\xb0\xe3\t\x92\xe4\x8fH\xe5\xa7a\x03\x1eo\x94\x